In [26]:
from bs4 import BeautifulSoup
from splinter import Browser
from selenium import webdriver
import time
import sys
import json
import dateparser 
import requests
import lxml
from lxml import html
from lxml import etree
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## Storing the Data

In [135]:
#We will be using a dictionary that resembles our main table. 
restaurants = {}
restaurants["names"] = []
restaurants["cusine"] = []
restaurants["price"] = []
restaurants["reviews_count"] = []
restaurants["rating"] = []
restaurants["address"] = []
restaurants["url"] = []



In [131]:
restaurants

{'names': ['Archipelago'],
 'cusine': ['Eclectic restaurant'],
 'price': ['£££'],
 'reviews_count': ['376 reviews'],
 'rating': ['4.5'],
 'address': ['53 Cleveland St, London W1T 4JJ'],
 'url': ['https://www.google.com/maps/place/Archipelago/@51.4848436,-0.164514,12z/data=!4m9!1m2!2m1!1srestaurants+in+london+uk!3m5!1s0x48761b28ff4c92f1:0xcc0da3fd3ada4401!8m2!3d51.5204806!4d-0.13881!15sChhyZXN0YXVyYW50cyBpbiBsb25kb24gdWtaGiIYcmVzdGF1cmFudHMgaW4gbG9uZG9uIHVrkgETZWNsZWN0aWNfcmVzdGF1cmFudA']}

## Access Google Maps

In [136]:
#Getting the base url
## Links in Google maps can be customized through queries: https://developers.google.com/maps/documentation/urls/get-started
###Forming the Search URL -> https://www.google.com/maps/search/?api=1&parameters
####Based on this, our base url is https://www.google.com/maps/search/?api=1&query=restaurants+in+london+uk
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.get('https://www.google.com/maps/search/?api=1&query=restaurants+in+london+uk')
time.sleep(5)
#The first pop up we get is to accept or customize cookies. Here, we will simply accept them by 
##clicking on the xpath of the 'agree' button
browser.find_element_by_xpath('//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div/div/button').click()
browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[7]/div/a').click()
time.sleep(3)
restaurants["names"].append(browser.find_element_by_class_name("x3AX1-LfntMc-header-title-title").text)
restaurants["cusine"].append(browser.find_element_by_xpath("""//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[2]/span[1]/span[1]/button""").text)
restaurants["price"].append(browser.find_elements_by_xpath("""//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span[2]/span[2]/span[1]/span""")[0].text)
restaurants["rating"].append(browser.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d").text)
restaurants["reviews_count"].append(browser.find_element_by_class_name("Yr7JMd-pane-hSRGPd").text)
restaurants["address"].append(browser.find_elements_by_class_name("QSFF4-text")[0].text)
restaurants["url"].append(browser.current_url)
#browser.find_element_by_xpath('//*[@id="omnibox-singlebox"]/div[1]/div[1]/button').click()
#browser.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[9]/div/a').click()
#restaurants["names"].append(browser.find_element_by_xpath("""//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]""").text)
browser.close()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/marianaarrieta/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # This is added back by InteractiveShellApp.init_path()
/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/h1/span[1]"}
  (Session info: chrome=98.0.4758.102)
Stacktrace:
0   chromedriver                        0x000000010b0dfee9 chromedriver + 5013225
1   chromedriver                        0x000000010b06b1d3 chromedriver + 4534739
2   chromedriver                        0x000000010ac41a68 chromedriver + 170600
3   chromedriver                        0x000000010ac76152 chromedriver + 385362
4   chromedriver                        0x000000010ac76311 chromedriver + 385809
5   chromedriver                        0x000000010aca8324 chromedriver + 590628
6   chromedriver                        0x000000010ac9380d chromedriver + 505869
7   chromedriver                        0x000000010aca6074 chromedriver + 581748
8   chromedriver                        0x000000010ac936d3 chromedriver + 505555
9   chromedriver                    